---

## Load US Cesnsus Data via API

In [2]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

In [5]:
# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the .env file
API_KEY = os.getenv('CENSUS_API_KEY')
BASE_URL = 'https://api.census.gov/data/2020/dec/pl'

# Define the parameters for the API request
params = {
    'get': 'NAME,P1_001N',  # P1_001N is the total population count
    'for': 'state:*',        # Get data for all states
    'key': API_KEY
}

# Make the API request
response = requests.get(BASE_URL, params=params)

if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()
    
    # Convert JSON data to a pandas DataFrame
    columns = data[0]
    data_rows = data[1:]
    df = pd.DataFrame(data_rows, columns=columns)
    
    # Display the first few rows of the DataFrame
    print(df.head(10))
else:
    print(f"Error: {response.status_code}")

# # Optional: save the DataFrame to a CSV file
# df.to_csv('census_population_by_state.csv', index=False)


                    NAME   P1_001N state
0           Pennsylvania  13002700    42
1             California  39538223    06
2          West Virginia   1793716    54
3                   Utah   3271616    49
4               New York  20201249    36
5   District of Columbia    689545    11
6                 Alaska    733391    02
7                Florida  21538187    12
8         South Carolina   5118425    45
9           North Dakota    779094    38
10                 Maine   1362359    23
11               Georgia  10711908    13
12               Alabama   5024279    01
13         New Hampshire   1377529    33
14                Oregon   4237256    41
15               Wyoming    576851    56
16               Arizona   7151502    04
17             Louisiana   4657757    22
18               Indiana   6785528    18
19                 Idaho   1839106    16


---

## Load Data from Google BigQuery

1. Set Up Google Cloud SDK and Authentication:
Ensure you have the Google Cloud SDK installed. If not, install it following the [installation guide](https://cloud.google.com/sdk/docs/install).

2. In the terminal, run the following command to execute the script:

`gcloud auth application-default login`


3. If not already installed, install the required packages

`pip install google-cloud-bigquery db-dtypes google-cloud-bigquery-storage`

In [3]:
from google.cloud import bigquery
import pandas as pd

# Create a BigQuery client
client = bigquery.Client(project='changemaker-data') # replace 'changemaker-data' with your project ID

# Define the query
query = """
SELECT *
FROM `bigquery-public-data.census_bureau_acs.congressionaldistrict_2020_5yr`
LIMIT 1000
"""

# Run the query and convert the results to a pandas DataFrame
query_job = client.query(query)  # Run the query
df = query_job.to_dataframe()    # Convert to pandas DataFrame

# Display the first few rows of the DataFrame
df.head()


C:\Users\david\anaconda3\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,geo_id,aggregate_travel_time_to_work,amerindian_including_hispanic,amerindian_pop,armed_forces,asian_including_hispanic,asian_male_45_54,asian_male_55_64,asian_pop,associates_degree,...,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,walked_to_work,white_including_hispanic,white_male_45_54,white_male_55_64,white_pop,worked_at_home,workers_16_and_over
0,1704,10998310.0,4120.0,304.0,124.0,26830.0,1588.0,1422.0,26012.0,25973.0,...,21374.0,5675.0,1549.0,10307.0,365945.0,9973.0,10363.0,159658.0,19364.0,340230.0
1,3607,12118075.0,3883.0,945.0,74.0,127411.0,8164.0,8019.0,126780.0,23863.0,...,22404.0,6275.0,1331.0,42154.0,338485.0,11795.0,8929.0,229998.0,28271.0,335728.0
2,3609,13974065.0,1860.0,796.0,152.0,52604.0,3552.0,2999.0,52053.0,33727.0,...,23311.0,6686.0,895.0,21860.0,261742.0,13823.0,12139.0,236346.0,26951.0,339198.0
3,3610,10787915.0,1894.0,807.0,44.0,139259.0,8214.0,7744.0,138514.0,19763.0,...,52269.0,11769.0,2380.0,72488.0,498490.0,26821.0,26134.0,454312.0,46315.0,376309.0
4,0200,6250965.0,107298.0,103843.0,18866.0,47289.0,3122.0,2653.0,45855.0,42162.0,...,63197.0,7555.0,2902.0,26789.0,466961.0,31060.0,35414.0,439979.0,21184.0,351387.0


In [4]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.wa.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.wa.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("f6w7-q2d2", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df.sample(3)

,vin_1_10,county,city,state,zip_code,model_year,make,model,ev_type,cafv_type,electric_range,base_msrp,legislative_district,dol_vehicle_id,geocoded_column,electric_utility,_2020_census_tract,:@computed_region_x4ys_rtnd,:@computed_region_fny7_vc3j,:@computed_region_8ddd_yn5v
701,1N4AZ1CP3K,King,Woodinville,WA,98072,2019,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,150,0,45,186047529,"{'type': 'Point', 'coordinates': [-122.1368031...",PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033021906,3009,1,47
219,YV4BR0DM1N,King,Seattle,WA,98122,2022,VOLVO,XC60,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,18,0,37,180300095,"{'type': 'Point', 'coordinates': [-122.3128637...",CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),53033007902,3009,9,4
1596,JTDKARFP1L,Clark,Vancouver,WA,98682,2020,TOYOTA,PRIUS PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,17,2500081,"{'type': 'Point', 'coordinates': [-122.5286031...",BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,53011041313,2977,3,24


---